# Installations

In [ ]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git #egg=sklearn_crfsuite

  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-req-build-t5s9f7qy
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-req-build-t5s9f7qy
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 9.7 MB/s eta 0:00:00
  Created wheel for sklearn-crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10868 sha256=c44e4e40b791902e23e469d460446545e8317b954a243abe890aafdea696e769
  Stored in directory: /tmp/pip-ephem-wheel-cache-efk1eaiu/wheels/0b/bc/07/bd75a6f5fa2bf2ea05a5aad8d9ac66d2b5aab93dfd4e1a89de
Successfully built sklearn-crfsuite


In [ ]:
!pip install sklearn-crfsuite

# Importing librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [ ]:
from tensorflow import keras
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from math import nan
from keras.optimizers import Optimizer

from tensorflow.keras.utils import to_categorical
# **Second**rflow_addons.layers import CRF


from subprocess import check_output

# Importing cleaned data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_cleaned.csv')

In [ ]:
df

,sentences,words,tag,label,sentence_id
0,menu opener,menu,O,other,0
1,menu opener,opener,O,other,0
2,our hotels,our,O,other,1
3,our hotels,hotel,O,other,1
4,back,back,O,other,2
...,...,...,...,...,...
77432,7.6,7.6,B-STAR,NB_STARS,23112
77433,7.6,7.6,B-STAR,NB_STARS,23113
77434,7.6,7.6,B-STAR,NB_STARS,23114
77435,7.6,7.6,B-STAR,NB_STARS,23115


# **Random Forest Model**

In [ ]:

import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
#from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
#from keras.models import Input

In [ ]:
# A class to retrieve the sentences from the dataset
class getsentence(object):

    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                           s["tag"].values.tolist(),
                                                           )]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [ ]:
getter = getsentence(df)
sentences = getter.sentences
#ths is how a sentence will look like.
print(sentences[1])

[('our', 'O'), ('hotel', 'O')]


In [ ]:
#Lets find the number of words in the dataset
words = list(set(df["words"].values))
n_words = len(words)
print(n_words)

4469


In [ ]:
from numpy import max

#Lets find out the longest sentence length in the dataset
maxlen = max([len(s) for s in sentences])
print ('Maximum sentence length:', maxlen)

Maximum sentence length: 25


In [ ]:
#Simple feature map to feed arrays into the classifier.
def feature_map(word):
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [ ]:
#We divide the dataset into train and test sets
words = [feature_map(w) for w in df["words"].values.tolist()]
tags = df["tag"].values.tolist()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

#Random Forest classifier
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),X=words, y=tags, cv=5)

In [ ]:
#Lets check the performance
from sklearn.metrics import classification_report
report = classification_report(y_pred=pred, y_true=tags)
print(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-CATTYPE       0.00      0.00      0.00      1296
      B-CURR       0.79      0.33      0.46      1207
       B-FPR       0.00      0.00      0.00        68
   B-HOTNAME       0.00      0.00      0.00      1204
      B-INCL       0.00      0.00      0.00        32
    B-NIGHTS       0.00      0.00      0.00      1200
       B-OCC       0.00      0.00      0.00      1205
        B-PR       0.09      0.04      0.06      1393
     B-PTYPE       0.00      0.00      0.00      1218
     B-RACAT       0.43      0.24      0.31      1241
     B-RNAME       0.02      0.00      0.00      1311
    B-RONAME       0.06      0.00      0.01      1301
       B-SLD       0.00      0.00      0.00        65
      B-STAR       0.70      0.97      0.82      1197
    B-TAXCAT       0.00      0.00      0.00        11
    B-TAXVAL       0.00      0.00      0.00         7
   I-CATTYPE       0.00      0.00      0.00      2450
      I-CURR       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **CRF Model**

In [ ]:
from itertools import chain

import nltk
import sklearn
import scipy.stats

import sklearn_crfsuite
from sklearn_crfsuite import scorers,CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics

In [ ]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [tag for token, tag in sent]

In [ ]:
#Creating the train and test set
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
X

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
y

[['O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O'],
 ['O'],
 ['O', 'O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

# Initialiser le modèle CRF
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score
#We predcit using the same 5 fold cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
#Lets evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-CATTYPE       1.00      1.00      1.00      1296
      B-CURR       1.00      1.00      1.00      1207
       B-FPR       1.00      0.82      0.90        68
   B-HOTNAME       0.65      1.00      0.79      1204
      B-INCL       0.89      1.00      0.94        32
    B-NIGHTS       0.93      1.00      0.96      1200
       B-OCC       1.00      1.00      1.00      1205
        B-PR       0.54      1.00      0.70      1393
     B-PTYPE       0.97      1.00      0.98      1218
     B-RACAT       1.00      0.44      0.61      1241
     B-RNAME       0.99      1.00      1.00      1311
    B-RONAME       1.00      1.00      1.00      1301
       B-SLD       1.00      0.85      0.92        65
      B-STAR       1.00      0.00      0.01      1197
    B-TAXCAT       0.00      0.00      0.00        11
    B-TAXVAL       0.00      0.00      0.00         7
   I-CATTYPE       1.00      1.00      1.00      2450
      I-CURR       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Tuning the parameters manually, setting c1 = 10
crf2 = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
pred = cross_val_predict(estimator=crf2, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-CATTYPE       1.00      1.00      1.00      1296
      B-CURR       1.00      0.88      0.94      1207
       B-FPR       1.00      0.90      0.95        68
   B-HOTNAME       1.00      0.01      0.02      1204
      B-INCL       1.00      1.00      1.00        32
    B-NIGHTS       0.83      1.00      0.91      1200
       B-OCC       1.00      1.00      1.00      1205
        B-PR       0.99      1.00      0.99      1393
     B-PTYPE       0.74      1.00      0.85      1218
     B-RACAT       1.00      1.00      1.00      1241
     B-RNAME       1.00      1.00      1.00      1311
    B-RONAME       1.00      1.00      1.00      1301
       B-SLD       1.00      0.89      0.94        65
      B-STAR       1.00      0.00      0.01      1197
    B-TAXCAT       0.00      0.00      0.00        11
    B-TAXVAL       0.00      0.00      0.00         7
   I-CATTYPE       1.00      1.00      1.00      2450
      I-CURR       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
crf2.fit(X, y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=10, c2=0.1,
    max_iterations=100)

In [ ]:
#Select all the tags that are relevant for us, remove the 'O' tag from the list.
labels = list(crf2.classes_)
labels = list(filter(lambda a: a != 'O', labels))
print(labels)

['B-CURR', 'B-NIGHTS', 'I-NIGHTS', 'B-OCC', 'I-OCC', 'B-HOTNAME', 'I-HOTNAME', 'B-STAR', 'I-STAR', 'B-TAXCAT', 'I-TAXCAT', 'B-INCL', 'I-INCL', 'B-RONAME', 'I-RONAME', 'B-RNAME', 'I-RNAME', 'B-PR', 'B-PTYPE', 'I-PTYPE', 'B-RACAT', 'B-CATTYPE', 'I-CATTYPE', 'B-FPR', 'I-FPR', 'I-CURR', 'B-SLD', 'I-PR', 'B-TAXVAL', 'I-RACAT']


In [ ]:
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats

#Creating the Randomized CV search model by using a modified F1 scorer model considering only the relevant labels


# define fixed parameters and parameters to search
crf3 = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X, y)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True, c1=0.1, c2=0.1,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c30130d86a0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c301339cbb0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-CURR', 'B-NIGHTS', 'I-NIGHTS', 'B-OCC', 'I-OCC', 'B-HOTNAME', 'I-HOTNAME', 'B-STAR', 'I-STAR', 'B-TAXCAT', 'I-TAXCAT', 'B-INCL', 'I-INCL', 'B-RONAME', 'I-RONAME', 'B-RNAME', 'I-RNAME', 'B-PR', 'B-PTYPE', 'I-PTYPE', 'B-RACAT', 'B-CATTYPE', 'I-CATTYPE', 'B-FPR', 'I-FPR', 'I-CURR', 'B-SLD', 'I-PR', 'B-TAXVAL', 'I-RACAT']),
                   verbose=1)

In [ ]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.2551262372067098, 'c2': 0.004144547971337484}
Best CV score: 0.9007464661987855
Model size: 0.01M


In [ ]:
#We sort the tags so that they appear in an orderly fashion in the classification report
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0]))

In [ ]:
#Now we create the model again using the best estimators
crf3 = rs.best_estimator_

y_pred = crf3.predict(X)
print(metrics.flat_classification_report(
    y, y_pred, labels=sorted_labels, digits=3))

In [ ]:
crf3.fit(X,y)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.2551262372067098,
    c2=0.004144547971337484, max_iterations=100)

In [ ]:
pip install eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107729 sha256=b3323308a15d9ccc0b27f40c63153e922bad79d231e20973adf0f84d4df39e99
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
import eli5
eli5.show_weights(crf2)

From \ To,O,B-CATTYPE,I-CATTYPE,B-CURR,I-CURR,B-FPR,I-FPR,B-HOTNAME,I-HOTNAME,B-INCL,I-INCL,B-NIGHTS,I-NIGHTS,B-OCC,I-OCC,B-PR,I-PR,B-PTYPE,I-PTYPE,B-RACAT,I-RACAT,B-RNAME,I-RNAME,B-RONAME,I-RONAME,B-SLD,B-STAR,I-STAR,B-TAXCAT,I-TAXCAT,B-TAXVAL
O,0.562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-CATTYPE,0.0,0.0,1.441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-CATTYPE,0.0,0.0,1.037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-CURR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-CURR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FPR,0.0,0.0,0.0,0.0,0.0,0.0,2.764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-FPR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-HOTNAME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-HOTNAME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-INCL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Entraîner le modèle CRF
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

**Evaluation**

**F1 Score**

In [ ]:
from sklearn_crfsuite.metrics import flat_f1_score

f1_score = flat_f1_score(y_test, y_pred, average='weighted')

f1_score

1.0

**Classification Report**

In [ ]:

report = metrics.flat_classification_report(y_test, y_pred,labels=sorted_labels)
print(report)

              precision    recall  f1-score   support

   B-CATTYPE       1.00      1.00      1.00       262
   I-CATTYPE       1.00      1.00      1.00       494
      B-CURR       1.00      1.00      1.00       248
      I-CURR       1.00      1.00      1.00        49
       B-FPR       1.00      1.00      1.00         9
       I-FPR       1.00      1.00      1.00        10
   B-HOTNAME       1.00      1.00      1.00       247
   I-HOTNAME       1.00      1.00      1.00       681
      B-INCL       1.00      1.00      1.00         7
      I-INCL       1.00      1.00      1.00         8
    B-NIGHTS       1.00      1.00      1.00       228
    I-NIGHTS       1.00      1.00      1.00       246
       B-OCC       1.00      1.00      1.00       242
       I-OCC       1.00      1.00      1.00       939
        B-PR       1.00      1.00      1.00       292
        I-PR       1.00      1.00      1.00       117
     B-PTYPE       1.00      1.00      1.00       250
     I-PTYPE       1.00    

**Confusion Matrix**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Prédire les étiquettes pour les données de test
y_pred = crf.predict(X_test)

# Obtenir la liste de toutes les étiquettes possibles
labels = list(crf.classes_)

# Calculer la matrice de confusion
cm = confusion_matrix(np.concatenate(y_test), np.concatenate(y_pred), labels=labels)

# Afficher la matrice de confusion
cm


array([[ 244,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  549,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 8134,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,  292,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,  262,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        

# **CRF after Cleaning**

In [ ]:

import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
#from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
#from keras.models import Input

**Create a list of sequences for each sentence in the DataFrame**

In [ ]:

import pandas as pd


word_sequences = []
tag_sequences = []


for sentence in df['sentences'].unique():

    # Sélectionner les mots et les tags pour cette phrase
    words = df.loc[df['sentences'] == sentence, 'words'].tolist()
    tags = df.loc[df['sentences'] == sentence, 'tag'].tolist()

    # Ajouter les séquences de mots et de tags à la liste correspondante
    word_sequences.append(words)
    tag_sequences.append(tags)


In [ ]:
word_sequences[530:540]

[['and',
  'of',
  'course',
  'no',
  'breakfast',
  'would',
  'be',
  'complete',
  'without',
  'a',
  'generous',
  'hot',
  'drink',
  'to'],
 ['start',
  'the',
  'day',
  'think',
  'coffee',
  'tea',
  'and',
  'hot',
  'chocolate',
  'and',
  'a',
  'refreshing'],
 ['selection', 'of', 'juice'],
 ['discover',
  'the',
  'delicious',
  'produce',
  'that',
  'make',
  'up',
  'our',
  'allyoucaneat',
  'breakfast'],
 ['buffet',
  'for',
  'yourself',
  'a',
  'well',
  'a',
  'the',
  'great',
  'value',
  'for',
  'money',
  'offered',
  'by',
  'the'],
 ['price',
  'and',
  'for',
  'the',
  'little',
  'one',
  'up',
  'to',
  '10',
  'year',
  'inclusive',
  'the',
  'full',
  'grownup'],
 ['breakfast', 'is', 'on', 'offer', 'for', 'a', 'mini', 'price'],
 ['escape',
  'to',
  'the',
  'west',
  'indie',
  'and',
  'sleep',
  'in',
  'our',
  '3star',
  'sea',
  'view',
  'hotel',
  'in',
  'fort'],
 ['defrance'],
 ['organise',
  'your',
  'holiday',
  'and',
  'discover',
  

In [ ]:
tag_sequences[530:540]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

**Creating Feature Set :**

In [ ]:
# @title
# Définir les caractéristiques pour chaque mot

def word_features(sentence, i):
  word = sentence[i]
  features = {
        'word': word,
        'is_first': i == 0,
        'is_last': i == len(sentence) - 1,
        #'is_capitalized': word[0].upper() == word[0] if len(word) > 0 else False,
        'is_all_caps': word.upper() == word if len(word) > 0 else False,
        'is_all_lower': word.lower() == word if len(word) > 0 else False,
        'prefix-1': word[0] if len(word) > 0 else '',
        'prefix-2': word[:2] if len(word) > 1 else '',
        'prefix-3': word[:3] if len(word) > 2 else '',
        'suffix-1': word[-1] if len(word) > 0 else '',
        'suffix-2': word[-2:] if len(word) > 1 else '',
        'suffix-3': word[-3:] if len(word) > 2 else '',
        'prev_word': '' if i == 0 else sentence[i-1],
        'next_word': '' if i == len(sentence)-1 else sentence[i+1],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
        'capitals_inside': word[1:].lower() != word[1:] if len(word) > 1 else False
    }
  return features

In [ ]:
# Définir les caractéristiques pour toutes les phrases

X = [ [word_features(word_seq, i) for i in range(len(word_seq))] for word_seq in word_sequences]
y = tag_sequences

In [ ]:
X[20]

[{'word': 'anger',
  'is_first': True,
  'is_last': True,
  'is_all_caps': False,
  'is_all_lower': True,
  'prefix-1': 'a',
  'prefix-2': 'an',
  'prefix-3': 'ang',
  'suffix-1': 'r',
  'suffix-2': 'er',
  'suffix-3': 'ger',
  'prev_word': '',
  'next_word': '',
  'has_hyphen': False,
  'is_numeric': False,
  'capitals_inside': False}]

In [ ]:

# Afficher les clés et les valeurs du dictionnaire
#for cle, valeur in X[0].items():
    #print(cle)


In [ ]:
# @title
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## **Hyperparameters Fine Tuning : Grid Search**

In [ ]:

'''from sklearn_crfsuite import CRF
from sklearn.model_selection import GridSearchCV

# Paramètres à ajuster
params_space = {
    'c1': [0.1, 0.5, 1.0],
    'c2': [0.1, 0.5, 1.0],
    'max_iterations': [50, 100, 200],
}

# Initialisation du modèle CRF
crf = CRF(algorithm='lbfgs', all_possible_transitions=True)

# Initialisation de l'objet GridSearchCV
grid = GridSearchCV(crf, params_space, cv=5, verbose=1)

# Ajustement des hyperparamètres sur les données d'entraînement
grid.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres et score associé
print('Meilleurs hyperparamètres:', grid.best_params_)
print('Score:', grid.best_score_)'''


**Best params :**

In [ ]:

import sklearn_crfsuite
from sklearn_crfsuite import metrics


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Cet argument spécifie l'algorithme à utiliser pour optimiser les poids du modèle. Ici, l'algorithme utilisé est L-BFGS, un algorithme d'optimisation quasi-Newtonien qui est bien adapté aux problèmes de classification séquentielle tels que la reconnaissance d'entités nommées (NER).
    c1=0.1, #Cet argument contrôle la force de la régularisation L1 appliquée aux poids du modèle. Une valeur plus élevée de c1 entraîne une régularisation plus forte et peut aider à prévenir le surapprentissage.
    c2=0.1, #Cet argument contrôle la force de la régularisation L2 appliquée aux poids du modèle. Comme pour c1, une valeur plus élevée de c2 entraîne une régularisation plus forte.
    max_iterations=100, #Cet argument spécifie le nombre maximal d'itérations à exécuter lors de l'optimisation des poids du modèle.
                        #Le nombre maximal d'itérations de l'algorithme d'entraînement à effectuer avant de s'arrête
    all_possible_transitions=True #Cet argument permet de spécifier si toutes les transitions entre les étiquettes doivent être prises en compte lors de l'entraînement du modèle ou seulement celles qui sont observées dans les données d'entraînement. Mettre cette valeur à True permet d'améliorer les performances du modèle en permettant l'apprentissage de transitions qui ne sont pas observées dans les données d'entraînement.
)

In [ ]:
# Entraîner le modèle CRF
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [ ]:

print('Accuracy of CRF classifier on training set: {:.2f}'
     .format(crf.score(X_train, y_train)))
print('Accuracy of CRF classifier on test set: {:.2f}'
     .format(crf.score(X_test, y_test)))

Accuracy of CRF classifier on training set: 0.99
Accuracy of CRF classifier on test set: 0.78


In [ ]:
y_test

[['O', 'O', 'O', 'O'],
 ['B-RONAME'],
 ['O'],
 ['O', 'O'],
 ['O', 'O'],
 ['B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B-STAR',
  'B

In [ ]:
# Tester le modèle sur l'ensemble de test
y_pred = crf.predict(X_test)

In [ ]:
y_pred

## **Evaluating the model performance**

**Confusion Matrix**

In [ ]:

import numpy as np
from sklearn.metrics import confusion_matrix

# Prédictions sur les données de test
y_pred = crf.predict(X_test)

# Conversion des séquences en listes pour la matrice de confusion
y_test_list = [elem for sublist in y_test for elem in sublist]
y_pred_list = [elem for sublist in y_pred for elem in sublist]


conf_matrix = confusion_matrix(y_test_list, y_pred_list)

print(conf_matrix)


[[  24    0    0    0    0    0    0    0    0    0  202    0    0    0
     0    0    0    0    0    0    0    0    0    0   14    0    0  155]
 [   0    0    0    0    0    0    0    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    4    0    0    0    0    0    3]
 [   0    0    7    0    0    0    0   10    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    1]
 [   0    0    0  102    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0   74]
 [   0    0    0    0    1    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    4]
 [   0    0    0    0    0   75    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0  305]
 [   0    0    0    0    0    0   40    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0  

In [ ]:
pip install --upgrade scikit-learn

**F1 Score**

In [ ]:
from sklearn_crfsuite.metrics import flat_f1_score

f1_score = flat_f1_score(y_test, y_pred, average='weighted')

f1_score

0.7734412512666669

**Classification Report**

In [ ]:
report = metrics.flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

   B-CATTYPE       1.00      0.64      0.78       550
      B-CURR       0.87      0.10      0.18       525
       B-FPR       0.50      0.33      0.40        15
   B-HOTNAME       0.89      0.88      0.89       125
    B-NIGHTS       1.00      0.24      0.39       100
       B-OCC       1.00      0.91      0.95        11
        B-PR       0.94      0.79      0.86       273
     B-PTYPE       1.00      0.88      0.94       414
     B-RACAT       0.95      0.97      0.96       672
     B-RNAME       0.85      0.46      0.60       280
    B-RONAME       0.96      0.73      0.83       224
       B-SLD       1.00      1.00      1.00        14
      B-STAR       0.89      1.00      0.94       421
   I-CATTYPE       0.85      0.81      0.83       821
      I-CURR       0.20      0.82      0.32        44
       I-FPR       0.50      0.31      0.38        16
   I-HOTNAME       0.44      0.96      0.60       378
    I-NIGHTS       1.00    

In [ ]:
# @title
y_test; y_pred

In [ ]:
sentences = ["room only", "hotel schweizerhof basel", "€92"]

In [ ]:

for sentence in sentences:
    word_sequence = sentence.split()
    X = [word_features(word_sequence, i) for i in range(len(word_sequence))]
    predicted_tags = crf.predict_single(X)
    print("Phrase : ", sentence)
    print("Mots : ", word_sequence)
    print("Tags prédits : ", predicted_tags)

Phrase :  room only
Mots :  ['room', 'only']
Tags prédits :  ['B-CATTYPE', 'I-CATTYPE']
Phrase :  hotel schweizerhof basel
Mots :  ['hotel', 'schweizerhof', 'basel']
Tags prédits :  ['O', 'O', 'O']
Phrase :  €92
Mots :  ['€92']
Tags prédits :  ['B-PR']
